In [ ]:
import sys
import os
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.stats import zscore

import cell2location
import scvi

from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text for PDFs

In [ ]:
root_path = os.getcwd()

In [ ]:
def significance_stars(p_value):
    """
    Generate significance stars based on p-value thresholds.
    
    Parameters:
        p_value (float): The p-value to evaluate.
        
    Returns:
        str: A string with significance stars ('', '*', '**', '***').
    """
    if p_value <= 0.001:
        return '***'
    elif p_value <= 0.01:
        return '**'
    elif p_value <= 0.05:
        return '*'
    else:
        return ''


In [ ]:
mylevel="celltype2"
results_folder = os.path.join(root_path, 'results/deconvolution_bbknn/'+mylevel+'_updated_v1/')

ref_run_name =  os.path.join(results_folder, 'reference_signatures') 
run_name = os.path.join(results_folder, 'cell2location_map')  

In [ ]:
adata_file = f"{run_name}/sp.h5ad"
adata_vis = sc.read_h5ad(adata_file)
# mod = cell2location.models.Cell2location.load(f"{run_name}", adata_vis)

In [ ]:
df_cellabundance = adata_vis.obsm['q05_cell_abundance_w_sf'].copy()

In [ ]:
adata_vis.uns['mod']['factor_names']

In [ ]:
df_cellabundance.columns =  adata_vis.uns['mod']['factor_names']

In [ ]:
df_merged  = pd.merge(adata_vis.obs, df_cellabundance,  left_index=True, right_index=True)

# A very first look to all the cell type abundances per condition (absolute values)

In [ ]:
# EDA: Plotting the distribution of a cell type across conditions
import seaborn as sns

for current_celltype in adata_vis.uns['mod']['factor_names']: 
    sns.boxplot(x='CONDITION', y=current_celltype, data=df_merged)
    plt.show()

In [ ]:
for current_celltype in adata_vis.uns['mod']['factor_names']: 
    sns.boxplot(x='CONDITION', y=current_celltype, data=df_merged, showfliers=False)
    plt.show()

## Working with proportions per spot

In [ ]:
row_sums = adata_vis.obsm['q05_cell_abundance_w_sf'].sum(axis=1)

In [ ]:
proportions_df = adata_vis.obsm['q05_cell_abundance_w_sf'].div(row_sums, axis=0)

In [ ]:
proportions_df.columns =  adata_vis.uns['mod']['factor_names']

In [ ]:
#proportions_df

In [ ]:
df_merged_prop = pd.merge(adata_vis.obs, proportions_df,  left_index=True, right_index=True)

In [ ]:
# EDA: Plotting the distribution of a cell type across conditions
import seaborn as sns

for current_celltype in adata_vis.uns['mod']['factor_names']: 
    sns.boxplot(x='CONDITION', y=current_celltype, data=df_merged_prop)
    plt.show()

In [ ]:
for current_celltype in adata_vis.uns['mod']['factor_names']: 
    sns.boxplot(x='CONDITION', y=current_celltype, data=df_merged_prop, showfliers=False)
    plt.show()

In [ ]:
from scipy.stats import mannwhitneyu

In [ ]:
df_cellabundance = adata_vis.obsm['q05_cell_abundance_w_sf'].copy()

In [ ]:
df_cellabundance_logTrans = np.log1p(df_cellabundance) 

In [ ]:
df_cellabundance_logTrans.columns =  adata_vis.uns['mod']['factor_names']

In [ ]:
#df_cellabundance_logTrans

In [ ]:
df_merged_LogTrans = pd.merge(adata_vis.obs, df_cellabundance_logTrans,  left_index=True, right_index=True)

In [ ]:
treati=[
 'V43J11-302_A1_B08',
'V43J19-050_A1_B07',    
 'V43J24-078_A1_B06',         
 'V43A13-374_A1_B19',         
 'V43A11-284_A1_B16']
controli=[
          'V43J11-302_D1_A02',
          'V43J19-319_D1_A03',
          'V43A11-284_D1_A04',
          'V43A13-374_D1_A05',
 'V43J19-050_D1_A08']

In [ ]:
spotnr=df_merged_prop.groupby('readout_id').sum('in_tissue')['in_tissue']

In [ ]:
import statsmodels.api as sm

In [ ]:
# Initialize a DataFrame to store the results
results_df = pd.DataFrame(columns=['Item', 'Coefficient', 'StdErr', 'CI_lower', 'CI_upper', 'P-value'])


In [ ]:

prop_df = pd.DataFrame(columns=['Item', 'Proportion', 'Treatment'])


In [ ]:
for current_celltype in adata_vis.uns['mod']['factor_names']: 

    #[df_merged_LogTrans['CONDITION'] == 'FAP_LTBR']
    group_treated=df_merged_prop.groupby('readout_id').mean(current_celltype).loc[treati,current_celltype] #[current_celltype].fillna(0)

    #[df_merged_LogTrans['CONDITION'] == 'FAP_LTBR']
    group_untreated=df_merged_prop.groupby('readout_id').mean(current_celltype).loc[controli,current_celltype] #[current_celltype].fillna(0)

    count1=[]
    nobs1=[]
    for mysample in list(group_treated.index):
        count1.append(int(group_treated[mysample]*spotnr.loc[mysample]))
        nobs1.append(int(spotnr.loc[mysample]))

    count2=[]
    nobs2=[]
    for mysample in list(group_untreated.index):
        count2.append(int(group_untreated[mysample]*spotnr.loc[mysample]))
        nobs2.append(int(spotnr.loc[mysample]))  


    # Combine counts and population sizes
    counts = np.concatenate((count1, count2))
    nobs = np.concatenate((nobs1, nobs2))
    treatment = np.concatenate((np.ones(len(count1)), np.zeros(len(count2))))

    # Save the proportions
    temp_df=pd.DataFrame([np.concatenate((group_treated, group_untreated)),treatment],index=['Proportion','Treatment']).transpose()
    temp_df['Item']=current_celltype
    prop_df=pd.concat([prop_df,temp_df])
    
    dependent_variable = np.column_stack((counts, nobs - counts))

    # Fit a GLM with a binomial family and a logit link function
    glm_binom = sm.GLM(dependent_variable, sm.add_constant(treatment), family=sm.families.Binomial())
    results = glm_binom.fit()

    # Extract the results
    coefficient = results.params[1]
    stderr = results.bse[1]
    ci_lower, ci_upper = results.conf_int()[1]
    p_value = results.pvalues[1]

    results_df = pd.concat([results_df,pd.DataFrame(pd.Series({
        'Item': current_celltype,
        'Coefficient': coefficient,
        'StdErr': stderr,
        'CI_lower': ci_lower,
        'CI_upper': ci_upper,
        'P-value': p_value
    })).transpose()])


In [ ]:
prop_df=prop_df.replace(1.0,'T').replace(0.0,'C')
results_df.index=list(results_df['Item'])
results_df=results_df.sort_values(by='Coefficient', ascending=False)


In [ ]:
figoutpath='/figures/'

In [ ]:
results_df.to_csv(os.path.join(figoutpath , 'ProportionTest_HighRes-FAP-LTBR_vs_Untreated-'+mylevel+'.tsv'),sep='\t')

In [ ]:
# Forest plot with significance stars and color coding
fig, ax = plt.subplots(figsize=(5, 6))

colors = ['royalblue' if x < 0 else 'coral' for x in results_df['Coefficient']]
for i in range(len(results_df)):
    ax.errorbar(results_df['Coefficient'].iloc[i], i, xerr=results_df['StdErr'].iloc[i], fmt='o', color=colors[i])
for i in range(len(results_df)):
    ax.hlines(i, results_df['CI_lower'].iloc[i], results_df['CI_upper'].iloc[i], colors='gray', lw=1)
ax.axvline(x=0, color='black', linestyle='--')
ax.set_yticks(range(len(results_df)))
ax.set_yticklabels(results_df['Item'], fontsize=8)
ax.invert_yaxis()
ax.set_xlabel('Log Odds Ratio')
for i, p_value in enumerate(results_df['P-value']):
    stars = significance_stars(p_value)
    #ax.text(results_df['Coefficient'].iloc[i], i, stars, ha='left', va='center')
    ax.text(results_df['Coefficient'].iloc[i], i-0.2, stars, 
            ha='center', va='center', color='black', fontsize=10)

plt.savefig(os.path.join(figoutpath , 'Proportions_HighRes-FAP-LTBR_vs_Untreated-'+mylevel+'.pdf'), format="pdf")
plt.show()


In [ ]:
prop_df.to_csv(os.path.join(figoutpath , 'ActualProportions_HighRes-FAP-LTBR_vs_Untreated-'+mylevel+'.tsv'),sep='\t')

In [ ]:
col_wrap_value = 5  # Adjust this value as needed

palette = {'T': 'coral', 'C': 'royalblue'}  # Choose your colors
g = sns.FacetGrid(prop_df, col='Item', sharey=False, height=4, aspect=0.9, col_wrap=col_wrap_value)
g.map(sns.boxplot, 'Treatment', 'Proportion', order=['C', 'T'], palette=palette)
g.map(sns.stripplot, 'Treatment', 'Proportion', order=['C', 'T'], color='black',
      dodge=True, jitter=False, alpha=0.5)
g.set_axis_labels('Treatment', 'Proportion')
g.set_titles('{col_name}')
plt.tight_layout()

plt.savefig(os.path.join(figoutpath , 'ActualProportions_HighRes-FAP-LTBR_vs_Untreated-'+mylevel+'.pdf'), format="pdf")
plt.show()


And if we go back to proportions as the intial morphology of the sample has an imapact. i.e. tumor areas can be larger in a particular sample regardless of being treated or not just because of the slide cut. 

In [ ]:
row_sums = adata_vis.obsm['q05_cell_abundance_w_sf'].sum(axis=1)

In [ ]:
proportions_df = adata_vis.obsm['q05_cell_abundance_w_sf'].div(row_sums, axis=0)

In [ ]:
proportions_df.columns =  adata_vis.uns['mod']['factor_names']

In [ ]:
proportions_df_logTrans = np.log1p(proportions_df) 

In [ ]:
df_merged_prop_LogTrans= pd.merge(adata_vis.obs, proportions_df_logTrans,  left_index=True, right_index=True)

In [ ]:
plt.hist(df_merged_prop_LogTrans['endothelial cell of high endothelial venule'], bins=60, alpha=0.7, color='blue')

In [ ]:
for current_celltype in adata_vis.uns['mod']['factor_names']: 
    
    group_treated = df_merged_prop_LogTrans[df_merged_prop_LogTrans['CONDITION'] == 'FAP_LTBR'][current_celltype]
    group_untreated = df_merged_prop_LogTrans[df_merged_prop_LogTrans['CONDITION'] == 'Untreated'][current_celltype]
    
    stat, p_value = mannwhitneyu(group_untreated, group_treated, alternative='two-sided')
    
    # Plotting
    sns.set(style="whitegrid")
    plt.figure(figsize=(10, 6))
    
    ax = sns.boxplot(x='CONDITION', y=current_celltype, data=df_merged_prop_LogTrans)
    
    max_value = df_merged_prop_LogTrans[current_celltype].max()
    ax.text(0.5, max_value, f'p-value={p_value:.3e}', 
        horizontalalignment='center', size='medium', color='black', weight='semibold')
    
    
    plt.title(str(current_celltype) + ' Log proportions by Condition')
    plt.show()

In [ ]:
for current_celltype in adata_vis.uns['mod']['factor_names']: 
    
    group_treated = df_merged_prop_LogTrans[df_merged_prop_LogTrans['CONDITION'] == 'FAP_LTBR'][current_celltype]
    group_untreated = df_merged_prop_LogTrans[df_merged_prop_LogTrans['CONDITION'] == 'Untreated'][current_celltype]
    
    stat, p_value = mannwhitneyu(group_untreated, group_treated, alternative='two-sided')
    
    # Plotting
    sns.set(style="whitegrid")
    plt.figure(figsize=(10, 6))
    
    ax = sns.boxplot(x='CONDITION', y=current_celltype, data=df_merged_prop_LogTrans, showfliers=False)
    
    max_value = 0
    ax.text(0.5, max_value, f'p-value={p_value:.3e}', 
        horizontalalignment='center', size='medium', color='black', weight='semibold')
    
    
    plt.title(str(current_celltype) + ' Log proportions by Condition')
    plt.show()

## Number per spots per sample as compared to the total number of spots under tissue with  proportions larger than a threshold

In [ ]:
df_merged_prop

In [ ]:
#2/40

In [ ]:
# Define the threshold: 1 cell out of 40 per spot. 
#threshold = 0.025
threshold = 0.025
filtered_df = df_merged_prop[df_merged_prop['endothelial cell of high endothelial venule'] > threshold]
# Group by 'readout_id' and count the number of records in each group
counts_above_threshold = filtered_df.groupby('readout_id').size()

In [ ]:
spotnr[treati]

In [ ]:
df_merged_prop[df_merged_prop[current_celltype] > threshold]['readout_id'].value_counts()[treati]

In [ ]:
# Initialize a DataFrame to store the results
results_df = pd.DataFrame(columns=['Item', 'Coefficient', 'StdErr', 'CI_lower', 'CI_upper', 'P-value'])
prop_df = pd.DataFrame(columns=['Item', 'Proportion', 'Treatment'])


for current_celltype in adata_vis.uns['mod']['factor_names']: 

    #[df_merged_LogTrans['CONDITION'] == 'FAP_LTBR']
    group_treated=df_merged_prop[df_merged_prop[current_celltype] > threshold]['readout_id'].value_counts()[treati]/spotnr.loc[treati] #[current_celltype].fillna(0)

    #[df_merged_LogTrans['CONDITION'] == 'FAP_LTBR']
    group_untreated=df_merged_prop[df_merged_prop[current_celltype] > threshold]['readout_id'].value_counts()[controli]/spotnr.loc[controli] #[current_celltype].fillna(0)

    count1=[]
    nobs1=[]
    for mysample in list(group_treated.index):
        count1.append(int(group_treated[mysample]*spotnr.loc[mysample]))
        nobs1.append(int(spotnr.loc[mysample]))

    count2=[]
    nobs2=[]
    for mysample in list(group_untreated.index):
        count2.append(int(group_untreated[mysample]*spotnr.loc[mysample]))
        nobs2.append(int(spotnr.loc[mysample]))  


    # Combine counts and population sizes
    counts = np.concatenate((count1, count2))
    nobs = np.concatenate((nobs1, nobs2))

    # Create an array to indicate treatment: 1 for treated, 0 for untreated
    treatment = np.concatenate((np.ones(len(count1)), np.zeros(len(count2))))

    # Save the proportions
    temp_df=pd.DataFrame([np.concatenate((group_treated, group_untreated)),treatment],index=['Proportion','Treatment']).transpose()
    temp_df['Item']=current_celltype
    prop_df=pd.concat([prop_df,temp_df])
    
    # The dependent variable in a binomial GLM is the number of successes and the number of failures
    # We create an array with two columns: the first column is the count of successes, the second is the count of failures
    dependent_variable = np.column_stack((counts, nobs - counts))

    # Fit a GLM with a binomial family and a logit link function
    glm_binom = sm.GLM(dependent_variable, sm.add_constant(treatment), family=sm.families.Binomial())
    results = glm_binom.fit()

    # Extract the results
    coefficient = results.params[1]
    stderr = results.bse[1]
    ci_lower, ci_upper = results.conf_int()[1]
    p_value = results.pvalues[1]


    # Append the results to the DataFrame
    results_df = pd.concat([results_df,pd.DataFrame(pd.Series({
        'Item': current_celltype,
        'Coefficient': coefficient,
        'StdErr': stderr,
        'CI_lower': ci_lower,
        'CI_upper': ci_upper,
        'P-value': p_value
    })).transpose()])



In [ ]:
results_df

In [ ]:
prop_df['Treatment']=prop_df['Treatment'].replace(1.0,'T').replace(0.0,'C')
results_df.index=list(results_df['Item'])
results_df=results_df.sort_values(by='Coefficient', ascending=False)



In [ ]:

# Forest plot with significance stars and color coding
fig, ax = plt.subplots(figsize=(5, 6))

# Determine the colors based on the sign of the coefficients
colors = ['royalblue' if x < 0 else 'coral' for x in results_df['Coefficient']]

# Plot the estimated effects with error bars and color coding
for i in range(len(results_df)):
    ax.errorbar(results_df['Coefficient'].iloc[i], i, xerr=results_df['StdErr'].iloc[i], fmt='o', color=colors[i])

# Plot the confidence intervals
for i in range(len(results_df)):
    ax.hlines(i, results_df['CI_lower'].iloc[i], results_df['CI_upper'].iloc[i], colors='gray', lw=1)

# Add a vertical line at 0 for reference
ax.axvline(x=0, color='black', linestyle='--')

# Set the y-axis labels to the sorted item names
ax.set_yticks(range(len(results_df)))
ax.set_yticklabels(results_df['Item'], fontsize=8)

# Invert the y-axis so that the largest effect size is at the top
ax.invert_yaxis()

# Set the x-axis label
ax.set_xlabel('Log Odds Ratio')

# Add significance stars next to the points
for i, p_value in enumerate(results_df['P-value']):
    stars = significance_stars(p_value)
    ax.text(results_df['Coefficient'].iloc[i], i, stars, ha='left', va='center')

# Display the plot
plt.show()

In [ ]:
# Specify the number of columns you want per row
col_wrap_value = 5  # Adjust this value as needed

# Define the color palette for the treated and untreated groups
palette = {'T': 'coral', 'C': 'royalblue'}  # Choose your colors

# Initialize a FacetGrid object with one row for each item
g = sns.FacetGrid(prop_df, col='Item', sharey=False, height=4, aspect=0.9, col_wrap=col_wrap_value)

# Map the boxplot to the FacetGrid
g.map(sns.boxplot, 'Treatment', 'Proportion', order=['C', 'T'], palette=palette)

# Map the stripplot to the FacetGrid to show the individual data points
# Adjust the 'jitter' parameter as needed for better visibility
g.map(sns.stripplot, 'Treatment', 'Proportion', order=['C', 'T'], color='black',
      dodge=True, jitter=False, alpha=0.5)

# Set axis labels and titles
g.set_axis_labels('Treatment', 'Proportion')
g.set_titles('{col_name}')

# Adjust the layout
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
df_total = pd.DataFrame(counts_above_threshold)

In [ ]:
df_total.reset_index(inplace = True)

In [ ]:
df_total_merge = df_total.merge(df_merged_prop, on='readout_id')

In [ ]:
df_total_merge

In [ ]:
ax = sns.boxplot(x='CONDITION', y=0, data=df_total_merge)
ax.set_ylabel('Spots with proportion of HEV  > 0.025')
plt.show()

In [ ]:
all_counts = df_merged_prop.groupby('readout_id').size()

In [ ]:
all_counts

In [ ]:
df_final = pd.DataFrame(counts_above_threshold/all_counts)

In [ ]:
df_final.reset_index(inplace = True)

In [ ]:
df_final_merge = df_final.merge(df_merged_prop, on='readout_id')

In [ ]:
ax = sns.boxplot(x='CONDITION', y=0, data=df_final_merge)
ax.set_ylabel('Corrected by initial spot size')
plt.show()

In [ ]:
! jupyter nbconvert --to html 08_03_Deconvolution_C2L_Proportions-updated.ipynb